In [1]:
import sys
import pandas as pd #import csv
import numpy as np
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QGridLayout, QFileDialog, QLabel, QStackedWidget
from PyQt5.QtCore import QCoreApplication
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5 import uic, QtGui
import math
from itertools import product

global type_to # 타입 변경을 위한 변수
global type_combo_res
global data # 비식별 전
global count # 데이터셋 중 수치형 데이터의 개수
global info_header
global preview
global top3
preview = 10 # 프리뷰로 보여줄 데이터 개수. 기존의 len(data)
info_header = ['이름', '최솟값', '최댓값', '평균', '중앙값', '표준편차', '최빈값'] # 수치형 info
type_combo_res=[]

In [2]:
# 비식별 기법
class anonymous() :
    def __init__(self,df):
        self.df = df
        self.backup = backup
    
    def delete_col(self, column):
        """
            delete_col : 열 삭제
            column : 삭제할 column명
        """
        self.backup = self[column]
        
        for i in range(len(self[column])):
            self[column].iloc[i] = "" # 공백으로 값 변경
            
        return after
    
    def delete_row(self, rows):
        """
            delete_row : 행 삭제
            - rows : 삭제할 row명
        """
        self.backup = self.iloc[rows]
        del_df = self.drop(rows, axis=0)
        return del_df
    
    # (종근) 마스킹 수정
    def masking(self,column,direction,number, max_len): # DF, column, 남겨둘 정보 위치(left/right), 남겨둘 정보 개수, column 내 데이터 중 가장 긴 길이
        self.backup = self[column]
        before_data=self.loc[:,column]   
        after = self.astype({column : 'object'})           #columns 데이터 타입 변경
        
        for i in range(len(before_data.index)):
            string = str(before_data.iloc[i])
            length = len(string)                  
            m_length = length - number            #마스킹할 문자 길이 구하기
            if(max_len>length):
                string = string +(" "*(max_len-length))
            length = len(string)
            
            if direction == 'left':             #남겨둘 정보(마스킹하지 않을 정보)의 위치=왼쪽
                if(length<number):
                    string ="*"*len(string)
                else:
                    string = string[:number]+'*'*len(string[number:])

            after.at[i,column] = string

        return after 
    
    # (민석) 상하단 코딩 수정
    def Drange(self,column,num):   #민석스 버전 drange      #(데이터프레임, 적용할 칼럼, 수의 간격)
        self.backup = self[column]
        before_data = self.loc[:,column]
        re_cut = pd.cut(before_data, num)
        after = self.astype({column : 'category'})
        self[column] = re_cut
        after = self
        return after

    def Rounding(self,column,loc,job):  #데이터프레임, 컬럼, 위치, 작업        
        if job==0:#반올림            
            self[column] = round(self[column], -loc)
            after=self
        return after
    
    def Pretreatment(self,column): # count =1 인 값 제거
        list2=[]
        list3=self[column].tolist()
        for i in list3:
            if(list3.count(i)==1):
                i = ''
                list2.append(i)
            else:
                list2.append(i)
        list3=list2
        self[column]=list3
        after=self
         
    def object_normalize(self,column): #문자형 데이터 전처리 
        s = pd.Series(self[column]).value_counts()
        s = (s-s.mean())/s.std()
        l_overvalue =s[s<-3].index
        result = l_overvalue
        for i in result:
            after= self.loc[self[column]!=i]
    
    #일반화
    def generation(self,column,level,delimeter) : # 구분자 delimeter
        listing = self[column] #레벨은 역순임. Level1이 제일 쌘 비식별이 됨
        try :
            a = list(map(lambda x : " ".join(x.split(delimeter)[i] for i in range(level)),listing)) #구분자로 자르고 ㅇ
            self[column] = a
            after = self
            return after
        except IndexError :
            print('레벨이 어울리지 않습니다.')

In [3]:
# 레벨트리 그리는 class
class LevelTree():
    def calculate(data, method, index):
        print("level tree 위해 calculate 하는 함수")
        #sample = data.iloc[:10, [index]] # 샘플로 10번째 행까지만 다룸
        sample = data.iloc[:, [index]]  # 실제 데이터 
        
        sample.rename(columns = {data.columns[index] : 'base'}, inplace = True) # column 명을 우선 임의로 지정해서
        max_len = sample.base.map(lambda x: len(str(x))).max() # data들 중 가장 긴 길이의 데이터 값 저장
        
        sample.rename(columns = {'base' : 'Level 0'}, inplace = True) # 원활한 출력을 위해 level 0으로 다시 컬럼명 변경
        level = [''] # level[0]을 비우고 싶어서 ...(?)
        levelTree = sample # return 할 levelTree에 우선 level 0을 넣어 선언
        original = sample
        print("level tree의 method" , method)
        if method.find('마스킹') != -1: # 기법이 마스킹이면
            for i, j in zip(reversed(range(max_len)), range(1, max_len+1)): # i = 3(역순으로) / j = 1 에서 시작
                level.append(anonymous.masking(sample, 'Level 0', 'left', i, max_len))
                level[j].rename(columns = {'Level 0' : 'Level '+str(j) }, inplace = True) # column명 변경
                levelTree = pd.concat([levelTree, level[j]], axis=1) # levelTree에 산출한 레벨 이어붙이기
        elif method.find('총계처리') != -1:
            col = data.iloc[:, [index]]
            sample['평균'] = col[data.columns[index]].mean()
            sample['중앙값'] = col[data.columns[index]].median()
            sample['최솟값'] = col[data.columns[index]].min()
            sample['최댓값'] = col[data.columns[index]].max()
            # total.rename(columns = {'Level 0' : 'Level 1' }, inplace = True) # column명 변경
            # levelTree = pd.concat([levelTree, level['Level 1']], axis=1) # levelTree에 산출한 레벨 이어붙이기
        elif method.find('라운딩') != -1: ###
            if '.' not in str(original.loc[0,'Level 0']) : ## 정수 실수 구분하기 위한 . 찾기
                for i, j in zip(range(1,max_len), range(1, max_len)) :
                    temp = original.copy()
                    a = anonymous.Rounding(temp, 'Level 0', i , 0) 
                    a.rename(columns = {'Level 0' : 'Level '+str(j) }, inplace = True)
                    levelTree = pd.concat([levelTree,a],axis=1)
            elif '.' in str(original.loc[0,'Level 0']) : ## 실수 임을 나타냄
                for i, j in zip( range(-3,max_len-3), range(1, max_len+1) ) : ## 소수점 셋째자리부터 라운딩 시작.
                    temp = original.copy()
                    a = anonymous.Rounding(temp, 'Level 0', i , 0)
                    a.rename(columns = {'Level 0' : 'Level '+str(j) }, inplace = True)
                    levelTree = pd.concat([levelTree,a],axis=1)
        elif method.find('상하단코딩') != -1:
            for i,j in zip( reversed(range(4,10)), range(1,7) ):
                temp = original.copy()
                a = anonymous.Drange(temp, 'Level 0', i)
                a.rename(columns = {'Level 0' : 'Level '+str(j) }, inplace = True)
                levelTree = pd.concat([levelTree,a],axis=1)
        elif method.find('일반화') != -1 :
            for i,j in zip(reversed(range(1,3)),range(1,3)):
                temp = original.copy()
                a = anonymous.generation(temp,'Level 0', i, " ")
                a.rename(columns = {'Level 0' : 'Level '+str(j) }, inplace = True)
                levelTree = pd.concat([levelTree,a],axis=1)

        return levelTree

In [4]:
# 전처리 클래스 
class pretreatment():
    def value_normalize(after,column):   #수치형 데이터만 준비 완료 누를 때 이상치 제거 
            s = after[column]
            s = s[((s-s.mean())/s.std()>3) | ((s-s.mean())/s.std()<-3)]
            after = after[~after[column].isin(s)]

In [5]:
# 타입 변경 ui            
type_change_class = uic.loadUiType("type_change.ui")[0]
class TypeChangeWindow(QDialog, type_change_class) :
    def __init__(self) :
        super().__init__()
        self.setupUi(self)
        self.yes_button.clicked.connect(self.change)
        
    def change(self) :
        global type_to
        if self.radio_object.isChecked() == True :
            self.close()
            type_to = 'object'
        elif self.radio_int.isChecked() == True :
            self.close()
            type_to = 'int64'
        elif self.radio_float.isChecked() == True :
            self.close()
            type_to = 'float64'

In [6]:
# level 정하는 window
class LevelWindow(QWidget):
    col = ""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        uic.loadUi('levelWindow_widget.ui', self)

        self.label_col.setText(self.col)

In [7]:
# delimiter 고를 수 있는 level window
class LevelWindow_partial(QWidget):
    col = ""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        uic.loadUi('levelWindow_partial_widget.ui', self)
        
        self.label_col.setText(self.col)

In [8]:
def brute(*args): # 입력한 준식별자에 대해서 점수 Top3 매기기
    global top3
    
    # ISO / IEC 문서 참고하여 점수 측정
    eee = {'로컬삭제' : {'S':0.5 ,'L':0.5,'I':0.5},
           '라운딩' : {'S':0 ,'L':0.5,'I':0.5},
           '마스킹' : {'S':1 ,'L':0.5,'I':0},
           '총계처리' : {'S':1 ,'L':1,'I':1},
           '상하단코딩' : {'S':0 ,'L':0.5,'I':0.5},
           '일반화' :{'S':0.5 ,'L':0.5,'I':0.5}
          }
        
    top3 = []
    bbb = list(product(*args, repeat = 1)) # 경우의수 (중복순열)
    list1 = []
        
    for i in bbb :
        sum1 = 0
        for j in i :
            sum1 +=sum(eee[j].values())
        list1.append(sum1 / len(type_combo_res)) # 해당 경우의 평균 점수를 list1에 저장
    
    for i in range(3) :
        top3.append(bbb[list1.index(max(list1))]) # list1에서 높은 점수 저장
        list1[list1.index(max(list1))] = 0 # 높은 점수 0으로 다음에 제외
    return top3 # 제일 높은 점수 3가지 경우 반환

In [9]:
# 비식별 적용 버튼 클릭 시, 새 창 띄워짐 (= anonymousWindow)
anonymous_class = uic.loadUiType("anonymousWindow_2.ui")[0]

class AnonymousWindow(QDialog, anonymous_class):
    def __init__(self) :
        super().__init__()
        self.setupUi(self)
        self.checked_method = [] # 기법 체크박스 값 리스트
        self.next_index = 2 # stackedWidget의 페이지 수 
        self.typelist = 0 # 있어 이런거..
        self.method = [] # methodBox에서 사용할 비식별 기법 리스트
        
        # 준식별자로 택한 column index
        global type_combo_res
        type_combo_res = np.array(type_combo_res) # list에서 해당 단어가 포함된 모든 index를 얻기 위해 numpy 사용
        self.QI_index = np.where(type_combo_res == '준식별자')[0] # 준식별자인 컬럼 index 리스트

        self.lineEdit_k.setValidator(QIntValidator(1,100,self)) # k 입력 받는 lineEdit => 1~100 정수만

        # 버튼 이벤트
        self.cancel_button.clicked.connect(self.closeWindow) # 취소 버튼 클릭 시, 창 닫힘
        self.next_button.clicked.connect(self.go_levelWindow) # next 버튼 클릭 시, addWidget한 위젯으로 이동할 수 있는 함수 호출
        
        # Window 이름 지정
        self.setWindowTitle("anonymous setting")
        
        # groupBox list 항목 -> horiziontal Layout 에 추가
        self.method_box_list = self.methodBox() # methodBox 함수를 이용해 groupbox 리스트를 받아옴
        for i in self.method_box_list :
            self.horizontalLayout_groupbox.addWidget(i) # groupbox list 하나씩 horizontalLayout에 addWidget 함
        
        self.show()
        
    def closeWindow(self):
        self.close()
    
    def methodBox(self): # 비식별 기법 체크박스 생성
        global data
        groupbox = [] # 준식별자에 적용할 checkbox의 groupbox를 담는 리스트
        j = 0 # groupbox index로 사용할 변수
        brutebox = []
        
        for i in self.QI_index:
            if data[data.columns[i]].dtype == 'object' :
                brutebox.append(['로컬삭제', '마스킹'])
            elif data[data.columns[i]].dtype == 'int64' or 'float64' :
                brutebox.append(['로컬삭제', '라운딩', '마스킹', '총계처리'])
            elif data[data.columns[i]].dtype == 'category' :
                brutebox.append(['로컬삭제', '마스킹', '일반화'])
                print('\nbrutebox에 일반화 들어갔나 확인', brutebox)
                
            groupbox_objName = "groupBox" + str(i)
            groupbox.append(QGroupBox(data.columns[i] + " (준식별자)", objectName=groupbox_objName)) # groupbox 제목 지정해서 groupbox list에 담음
            
            self.method = ['로컬삭제', '라운딩', '마스킹', '총계처리', '상하단코딩', '일반화'] # 비식별 기술 리스트 
            chkbox_list = [] # 각 groupbox에 들어갈 checkbox 리스트

            check_idx = 0
            for item in self.method:
                checkbox_objName = "checkBox_" + str(i) + "_" + str(check_idx) 
                chkbox_list.append(QCheckBox(item, objectName=checkbox_objName)) # 체크박스 리스트에 각 체크박스 항목을 넣는 for문
                check_idx += 1
            vbox = QVBoxLayout() 
            for chkbox in chkbox_list:
                vbox.addWidget(chkbox) # box layout에 체크박스들을 담은 리스트를 addWidget함
            groupbox[j].setLayout(vbox)
            groupbox[j].setLayout
            j+=1 # groupbox index 증가
            
        brute(*brutebox) # 비식별 추천
        self.label_2.setText(str(top3[0]))
        self.label_3.setText(str(top3[1]))
        self.label_4.setText(str(top3[2]))
        return groupbox
    
    def go_levelWindow(self):
        global data    
        
        if self.typelist == 0:
            self.getTypeList() # getTypeList 함수는 한 번만 실행되게 !
            k = self.lineEdit_k.text() # k 입력받음
            self.typelist += 1
        
        print("checked_method : ", self.checked_method)
        print("QI_index : ", self.QI_index)
        
        """ 
            - stackedWidget 추가하는 과정
             참고 ! index 0 : 기본 k 입력 받고 비식별 기법 적용하는 위젯
                    index 1 : 빈칸
                    index 2 : addWidget 한 내용
        """
        for method, qi in zip(self.checked_method, self.QI_index):
            if method.find('부분 삭제') != -1: # 부분삭제 비식별기법 사용 시
                self.levelWindow_p = LevelWindow_partial()
                self.levelWindow_p.label_col.setText(data.columns[qi]) # 레벨트리 화면 제목 지정
                self.stackedWidget.addWidget(self.levelWindow_p)
            else : # 그 외의 비식별기법 사용 시,
                self.levelWindow = LevelWindow()
                self.levelWindow.label_col.setText(data.columns[qi]) # 레벨트리 화면 제목 지정
                level = LevelTree.calculate(data, method, qi)
                
                self.levelWindow.tableWidget.setRowCount(preview) # data type, 개인정보 유형 -> "2행" 이후 데이터 출력
                self.levelWindow.tableWidget.setColumnCount(len(list(level[:10])))  # 10행만 출력
                self.levelWindow.tableWidget.setHorizontalHeaderLabels(list(level[:10]))
                
                
                
                
                
                for i in range(preview) :
                    for j in range (len(list(level))) :
                         self.levelWindow.tableWidget.setItem(i, j, QTableWidgetItem(format(level.values.tolist()[i][j]))) # 개인정보 유형, 데이터타입 -> "2행" 이후 데이터 출력
                self.stackedWidget.addWidget(self.levelWindow)
                
            data[data.columns[qi]] = '*'  #테스트 코드 *대신 비식별 값
            
            
        if self.next_index == 1+len(self.QI_index): # 마지막 stackedWidget일 때, 
            self.next_button.setText('완료') # 다음 버튼 대신 완료 버튼으로 변경
            self.next_button.clicked.connect(self.complete) # 완료 버튼 클릭 시, complete 함수 실행
                
        # stackedWidget index 이동
        print(self.next_index,"번째 stackedWidget")
        self.stackedWidget.setCurrentIndex(self.next_index)
        self.next_index += 1 # 다음버튼 클릭 후 그 다음에 다음버튼할 index
    
    def complete(self):
        self.WindowClass = WindowClass()  ## 비식별된 데이터 table_2에 추가   (김종근 코드 테스트 중 )
        print("완료 값 ",data)
        
        row = [*range(1, preview + 1)]
        row.insert(0,"타입")
        row = list(map(str, row))
        
        self.WindowClass.table_2.setRowCount(preview + 1) # data type, 개인정보 유형 -> "2행" 이후 데이터 출력
        self.WindowClass.table_2.setColumnCount(len(list(data)))
        self.WindowClass.table_2.setHorizontalHeaderLabels(list(data))
        self.WindowClass.table_2.setVerticalHeaderLabels(row)
        self.close()
    
    def getTypeList(self): # 체크된 기법 리스트에 담는 과정     
        for i in self.QI_index: # method_box_list의 모든 그룹 박스
            for j in range(len(self.method)): # 체크박스 (0~5 = range(6))
                checkbox_name = "checkBox_" + str(i) + "_" + str(j)
                chkbox = self.findChild(QCheckBox, checkbox_name)
                if chkbox.isChecked() : self.checked_method.append(chkbox.text()) 

In [10]:
form_class = uic.loadUiType("4thUI2.ui")[0]

#화면을 띄우는데 사용되는 Class 선언
class WindowClass(QMainWindow, form_class) :    
    def __init__(self) :
        super().__init__()
        self.setupUi(self)
        
        # 로고 이미지
        qPixmapVar = QPixmap()
        qPixmapVar.load("../인터페이스/logo2.jpg")
        self.Logo.setPixmap(qPixmapVar)
        
        # 버튼 이벤트
        self.open_button.clicked.connect(self.open_file) # open_button 클릭 관련
        self.apply_button.clicked.connect(self.open_anonymousWindow) # '비식별 처리' 버튼 클릭 관련
        self.save_button.clicked.connect(self.save_file) # save_button 클릭 관련
        self.type_button.clicked.connect(self.type_change) # 타입 변경
        self.set_button.clicked.connect(self.set_after) # --
        
        self.loc = 0
        self.job = 0
        loc = self.loc
        
    def open_file(self) : # 파일 불러오기
        global data
        global count
        global info_header
        
        open_name = QFileDialog.getOpenFileName(self, 'Open File', './')       
        if (max(open_name) != ""):
            data = pd.read_csv(open_name[0], encoding='utf-8')
            self.status.setText(open_name[0])

            global after # 비식별 처리 후 데이터를 저장할 전역변수 after
            after = data
            count = list()
            
            # 개인정보 유형(식별자, 준식별자, 민감정보, 비민감정보) Combo Box
            global type_combo_list # 개인정보 유형 콤보박스들을 담는 리스트
            type_combo_list = []
            for i in range(len(data.columns)):
                type_combo = QComboBox()
                type_combo.addItems(['비민감정보','식별자', '준식별자', '민감정보'])
                type_combo_list.append(type_combo)
            
            # row header 추가하기 위한 list 생성
            row = [*range(1, preview + 2)]
            row.insert(0,"타입")
            row.insert(0,"")
            row = list(map(str, row))
            
            # table_1 widget 행, 열 추가
            self.table_1.setRowCount(preview + 2) # data type, 개인정보 유형 -> "2행" 이후 데이터 출력
            self.table_1.setColumnCount(preview + 1)
            self.table_1.setHorizontalHeaderLabels(list(data))
            self.table_1.setVerticalHeaderLabels(row)

            # table_1 widget에 개인정보 유형 콤보박스 추가
            for i in range(len(data.columns)):
                self.table_1.setCellWidget(0, i, type_combo_list[i]) # 0번째 행에 개인정보 유형 Combo Box
            
            # 비식별 전 table_1에 
            # table_3 widget에 info추가 하기 위해서 수치형 개수 count
            for i in range(len(data.columns)) :
                if data[data.columns[i]].dtype == 'object' :
                    self.table_1.setItem(1, i, QTableWidgetItem('문자열')) # 1번째 행에 데이터 타입
                elif data[data.columns[i]].dtype == 'int64' :
                    self.table_1.setItem(1, i, QTableWidgetItem('정수')) # 1번째 행에 데이터 타입
                elif data[data.columns[i]].dtype == 'float64' :
                    self.table_1.setItem(1, i, QTableWidgetItem('실수')) # 1번째 행에 데이터 타입
                
                if (data[data.columns[i]].dtype == 'int64') | (data[data.columns[i]].dtype == 'float64') :
                    count.append(i)

            # table_1 widget에 불러온 파일의 데이터 추가
            for i in range (preview) :
                for j in range (len(list(data))) :
                    self.table_1.setItem(i + 2, j, QTableWidgetItem(format(data.values.tolist()[i][j]))) # 개인정보 유형, 데이터타입 -> "2행" 이후 데이터 출력
            
            # 수치형 데이터 table_3에 min, max ,median 등
            self.table_3.setRowCount(len(count))
            self.table_3.setColumnCount(len(list(info_header)))
            self.table_3.setHorizontalHeaderLabels(list(info_header))
            
            # 수치형 데이터의 이름, 최솟값, 최댓값, 평균, 중앙값, 표준편차, 최빈값 출력
            for i in range(len(count)) :
                self.table_3.setItem(i, 0, QTableWidgetItem(format(data.columns[count[i]])))
                self.table_3.setItem(i, 1, QTableWidgetItem(format(data[data.columns[count[i]]].min())))
                self.table_3.setItem(i, 2, QTableWidgetItem(format(data[data.columns[count[i]]].max())))
                self.table_3.setItem(i, 3, QTableWidgetItem("%.2lf"%(data[data.columns[count[i]]].mean())))
                self.table_3.setItem(i, 4, QTableWidgetItem(format(data[data.columns[count[i]]].median())))
                self.table_3.setItem(i, 5, QTableWidgetItem("%.3lf"%(data[data.columns[count[i]]].std())))
                self.table_3.setItem(i, 6, QTableWidgetItem(format(data[data.columns[count[i]]].value_counts().index[0])))
        else:
            print('파일 선택 안함')
            
    # 타입 변경
    def type_change(self) :
        global type_to
        global data
        
        type_change_Window = TypeChangeWindow()
        type_change_Window.exec_()
        column = self.table_1.currentColumn()
        data[data.columns[column]] = data[data.columns[column]].astype(type_to)
        for i in range(len(data.columns)) :
                if data[data.columns[i]].dtype == 'object' :
                    self.table_1.setItem(1, i, QTableWidgetItem('문자열')) # 1번째 행에 데이터 타입
                elif data[data.columns[i]].dtype == 'int64' :
                    self.table_1.setItem(1, i, QTableWidgetItem('정수')) # 1번째 행에 데이터 타입
                elif data[data.columns[i]].dtype == 'float64' :
                    self.table_1.setItem(1, i, QTableWidgetItem('실수')) # 1번째 행에 데이터 타입
    
    def set_after(self) :
        global type_combo_list
        global type_combo_res
        global count
        global info_header
        
        type_combo_res = [] # 개인정보 유형 콤보박스 선택 값 넣는 리스트
        for combo in type_combo_list:
            type_combo_res.append(combo.currentText()) # 현재 콤보박스 text를 type_combo_res에 append

        # 수치형 전처리 
        for i in range(len(after.columns)):
            if type_combo_res[i] == '식별자' :
                after[after.columns[i]] = ''
            if(after[after.columns[i]].dtype != object):
                pretreatment.value_normalize(after, after.columns[i])
            else:
                pass
            

        # row header 추가하기 위한 list 생성
        row = [*range(1, preview + 1)]
        row.insert(0,"타입")
        row = list(map(str, row))
        
        self.table_2.setRowCount(preview + 1) # data type, 개인정보 유형 -> "2행" 이후 데이터 출력
        self.table_2.setColumnCount(len(list(after)))
        self.table_2.setHorizontalHeaderLabels(list(after))
        self.table_2.setVerticalHeaderLabels(row)
        
        # table_2 widget에 불러온 파일의 데이터 추가
        for i in range (preview) :
                for j in range (len(list(after))) :
                    self.table_2.setItem(i + 1, j, QTableWidgetItem(format(after.values.tolist()[i][j]))) # 개인정보 유형, 데이터타입 -> "2행" 이후 데이터 출력
        
        # table_2 widget에 데이터 타입 추가
        for i in range(len(after.columns)) :
                if after[after.columns[i]].dtype == 'object' :
                    self.table_2.setItem(0, i, QTableWidgetItem('문자열')) # 1번째 행에 데이터 타입
                elif after[after.columns[i]].dtype == 'int64' :
                    self.table_2.setItem(0, i, QTableWidgetItem('정수')) # 1번째 행에 데이터 타입
                elif after[after.columns[i]].dtype == 'float64' :
                    self.table_2.setItem(0, i, QTableWidgetItem('실수')) # 1번째 행에 데이터 타입
        
        # 수치형 데이터 table_3에 min, max ,median 등
        self.table_4.setRowCount(len(count))
        self.table_4.setColumnCount(len(list(info_header)))
        self.table_4.setHorizontalHeaderLabels(list(info_header))

        # 수치형 데이터의 이름, 최솟값, 최댓값, 평균, 중앙값, 표준편차, 최빈값 출력
        for i in range(len(count)) :
            self.table_4.setItem(i, 0, QTableWidgetItem(format(after.columns[count[i]])))
            self.table_4.setItem(i, 1, QTableWidgetItem(format(after[after.columns[count[i]]].min())))
            self.table_4.setItem(i, 2, QTableWidgetItem(format(after[after.columns[count[i]]].max())))
            self.table_4.setItem(i, 3, QTableWidgetItem("%.2lf"%(after[after.columns[count[i]]].mean())))
            self.table_4.setItem(i, 4, QTableWidgetItem(format(after[after.columns[count[i]]].median())))
            self.table_4.setItem(i, 5, QTableWidgetItem("%.3lf"%(after[after.columns[count[i]]].std())))
            self.table_4.setItem(i, 6, QTableWidgetItem(format(after[after.columns[count[i]]].value_counts().index[0])))

    def open_anonymousWindow(self):
        # 준식별자 값 전달 필요
        anonymousWindow = AnonymousWindow()
        anonymousWindow.exec_()
        anonymousWindow.show()
                
    def save_file(self) : # 비식별처리 후 파일 저장
        save_name = QFileDialog.getSaveFileName(self, 'Open File', './')
        after.to_csv(save_name[0], encoding='utf-8', index=False)
        
if __name__ == "__main__" :
    app = QApplication(sys.argv) 
    myWindow = WindowClass()
    myWindow.show()
    app.exec_()

checked_method :  ['마스킹', '라운딩']
QI_index :  [0 3]
level tree 위해 calculate 하는 함수
level tree의 method 마스킹
level tree 위해 calculate 하는 함수
level tree의 method 라운딩
2 번째 stackedWidget


C:\Users\dgd\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-2-a8e7031269ff>:30: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self.backup = self[column]


checked_method :  ['마스킹', '라운딩']
QI_index :  [0 3]
level tree 위해 calculate 하는 함수
level tree의 method 마스킹
level tree 위해 calculate 하는 함수
level tree의 method 라운딩
3 번째 stackedWidget
checked_method :  ['마스킹', '라운딩']
QI_index :  [0 3]
level tree 위해 calculate 하는 함수
level tree의 method 마스킹
level tree 위해 calculate 하는 함수
level tree의 method 라운딩
4 번째 stackedWidget
완료 값     name                     email          phone age sex       address family  \
0     *    seonghyeon32@naver.com  010-8718-1075   *   F      전라북도 부안군   1남1녀   
1     *            yyang@nate.com  010-7999-1725   *   M      경상남도 함안군     편모   
2     *    eunjeong71@hanmail.net  010-8148-6815   *   F      충청북도 보은군     편부   
3     *       jaeho39@hanmail.net  010-8543-8719   *   M      충청북도 괴산군     편부   
4     *      gyeonghyi57@daum.net  010-5150-3215   *   F      울산광역시 북구   2남1녀   
5     *       subin70@hotmail.com  010-4132-4678   *   F      충청남도 당진군   실부실모   
6     *      munyeeun@hanmail.net  010-1537-6434   *   M      전라북도 무주군   1남

TypeError: Could not convert ******************** to numeric